In [8]:

import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import json
from datetime import datetime, timezone

def weatherapi(latitude, longitude,location_name):
    # Setup session with caching and retry logic
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    client = openmeteo_requests.Client(session=retry_session)

    # API endpoint and parameters
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature",
            "precipitation_probability", "precipitation", "rain", "showers", "snowfall",
            "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover",
            "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility",
            "evapotranspiration", "et0_fao_evapotranspiration", "vapour_pressure_deficit"
        ]
    }

    # Fetch weather data
    responses = client.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()

    # Generate datetime range for all hourly data points
    time_range = pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )

    # Find current UTC hour rounded down
    now_utc = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)

    # Find the index of the current hour in the time_range
    try:
        current_index = time_range.get_loc(now_utc)
    except KeyError:
        # If current time not in forecast range, fallback to nearest time (first available)
        current_index = 0

    # Prepare data for the current hour only
    current_hour_data = {"location": location_name,"date": now_utc.strftime("%Y-%m-%d %H:%M:%S")}

    variables = [
        "temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature",
        "precipitation_probability", "precipitation", "rain", "showers", "snowfall",
        "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover",
        "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility",
        "evapotranspiration", "et0_fao_evapotranspiration", "vapour_pressure_deficit"
    ]

    # Extract the value for the current hour from each variable
    for i, var in enumerate(variables):
        values = hourly.Variables(i).ValuesAsNumpy()
        current_hour_data[var] = float(values[current_index]) if values.size > current_index else None

    return current_hour_data

def fetch_weather_batch():
    delhi=weatherapi(28.7041,77.1025,'delhi')

    mumbai=weatherapi(18.9582,72.8321,'mumbai')

    bengaluru=weatherapi(12.9629,77.5775,'bengaluru')

    hyderabad=weatherapi(17.4065, 78.4772,'hyderabad')

    chennai=weatherapi(13.0843,80.2705,'chennai')

    kolkata=weatherapi(22.5744,88.3629,'kolkata')

    ahmedabad=weatherapi(23.0225,72.5714,'ahmedabad')

    pune=weatherapi(18.5246,73.8786,'pune')

    jaipur=weatherapi(26.9124,75.7873,'jaipur')

    lucknow=weatherapi(26.8467,80.9462,'lucknow')
    
    all_cities=[delhi,mumbai,bengaluru,hyderabad,chennai,kolkata,ahmedabad,pune,jaipur,lucknow]

    # debugging
    print(all_cities)


    return all_cities

fetch_weather_batch()











"""
    Fetches current hourly weather data for a specified location using the Open-Meteo API.
    
    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        location_name (str): Human-readable name of the location (for reference).
        
    Returns:
        dict: Weather data for the current hour, including temperature, humidity, precipitation,
              cloud cover, and other atmospheric variables.
              
    Implementation details:
        - Uses a cached session with automatic retries to improve API request reliability.
        - Fetches hourly forecast data from Open-Meteo API for a wide range of weather variables.
        - Converts API response timestamps to pandas datetime for easier time handling.
        - Extracts the data corresponding to the current UTC hour.
        - Returns a dictionary with all requested weather parameters for the current hour.

        
Data Dictionary Explanation:

- location: Name of the city (string).
- date: Date and time of the weather reading in 'YYYY-MM-DD HH:MM:SS' format (string).
- temperature_2m: Air temperature measured 2 meters above the ground, in degrees Celsius (float).
- relative_humidity_2m: Relative humidity at 2 meters height, in percentage (%) (float).
- dew_point_2m: Dew point temperature at 2 meters height, in degrees Celsius (float).
- apparent_temperature: Feels-like temperature considering humidity and wind, in degrees Celsius (float).
- precipitation_probability: Probability of precipitation occurring, in percentage (%) (float).
- precipitation: Amount of precipitation expected or recorded, in millimeters (mm) (float).
- rain: Amount of rain specifically, in millimeters (mm) (float).
- showers: Amount of showers, in millimeters (mm) (float).
- snowfall: Amount of snowfall, in millimeters (mm) (float).
- snow_depth: Depth of snow on the ground, in millimeters (mm) (float).
- weather_code: Numerical code representing weather conditions (e.g., clear, rain, snow) (float).
- pressure_msl: Atmospheric pressure at mean sea level, in hectopascals (hPa) (float).
- surface_pressure: Atmospheric pressure at the surface, in hectopascals (hPa) (float).
- cloud_cover: Total cloud cover percentage (%) (float).
- cloud_cover_low: Low-level cloud cover percentage (%) (float).
- cloud_cover_mid: Mid-level cloud cover percentage (%) (float).
- cloud_cover_high: High-level cloud cover percentage (%) (float).
- visibility: Visibility distance in meters (float).
- evapotranspiration: Amount of evapotranspiration, representing water transfer from land to atmosphere, in millimeters (mm) (float).
- et0_fao_evapotranspiration: Reference evapotranspiration (FAO standard), in millimeters (mm) (float).
- vapour_pressure_deficit: Difference between saturation and actual vapor pressure, indicating dryness of air, in kilopascals (kPa) (float).

"""


[{'location': 'delhi', 'date': '2025-06-23 21:00:00', 'temperature_2m': 27.62849998474121, 'relative_humidity_2m': 88.0, 'dew_point_2m': 25.459701538085938, 'apparent_temperature': 34.185821533203125, 'precipitation_probability': 43.0, 'precipitation': 0.30000001192092896, 'rain': 0.0, 'showers': 0.30000001192092896, 'snowfall': 0.0, 'snow_depth': 0.0, 'weather_code': 80.0, 'pressure_msl': 995.5, 'surface_pressure': 970.77099609375, 'cloud_cover': 86.0, 'cloud_cover_low': 78.0, 'cloud_cover_mid': 30.0, 'cloud_cover_high': 5.0, 'visibility': 24140.0, 'evapotranspiration': 0.029999999329447746, 'et0_fao_evapotranspiration': 0.0, 'vapour_pressure_deficit': 0.4433712959289551}, {'location': 'mumbai', 'date': '2025-06-23 21:00:00', 'temperature_2m': 27.614999771118164, 'relative_humidity_2m': 88.0, 'dew_point_2m': 25.4464168548584, 'apparent_temperature': 30.897842407226562, 'precipitation_probability': 100.0, 'precipitation': 1.5, 'rain': 0.10000000149011612, 'showers': 1.399999976158142, 

"\n    Fetches current hourly weather data for a specified location using the Open-Meteo API.\n    \n    Args:\n        latitude (float): Latitude of the location.\n        longitude (float): Longitude of the location.\n        location_name (str): Human-readable name of the location (for reference).\n        \n    Returns:\n        dict: Weather data for the current hour, including temperature, humidity, precipitation,\n              cloud cover, and other atmospheric variables.\n              \n    Implementation details:\n        - Uses a cached session with automatic retries to improve API request reliability.\n        - Fetches hourly forecast data from Open-Meteo API for a wide range of weather variables.\n        - Converts API response timestamps to pandas datetime for easier time handling.\n        - Extracts the data corresponding to the current UTC hour.\n        - Returns a dictionary with all requested weather parameters for the current hour.\n\n        \nData Dictionary E

In [ ]:
import pandas as pd
from datetime import datetime

#convert to dataframe
df = pd.DataFrame(fetch_weather_batch())


[{'location': 'delhi', 'date': '2025-06-23 21:00:00', 'temperature_2m1': 27.62849998474121, 'relative_humidity_2m': 88.0, 'dew_point_2m': 25.459701538085938, 'apparent_temperature': 34.185821533203125, 'precipitation_probability': 43.0, 'precipitation': 0.30000001192092896, 'rain': 0.0, 'showers': 0.30000001192092896, 'snowfall': 0.0, 'snow_depth': 0.0, 'weather_code': 80.0, 'pressure_msl': 995.5, 'surface_pressure': 970.77099609375, 'cloud_cover': 86.0, 'cloud_cover_low': 78.0, 'cloud_cover_mid': 30.0, 'cloud_cover_high': 5.0, 'visibility': 24140.0, 'evapotranspiration': 0.029999999329447746, 'et0_fao_evapotranspiration': 0.0, 'vapour_pressure_deficit': 0.4433712959289551}, {'location': 'mumbai', 'date': '2025-06-23 21:00:00', 'temperature_2m1': 27.614999771118164, 'relative_humidity_2m': 88.0, 'dew_point_2m': 25.4464168548584, 'apparent_temperature': 30.897842407226562, 'precipitation_probability': 100.0, 'precipitation': 1.5, 'rain': 0.10000000149011612, 'showers': 1.399999976158142

In [7]:
df

,location,date,temperature_2m1,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,...,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit
0,delhi,2025-06-23 21:00:00,27.628500,88.0,25.459702,34.185822,43.0,0.3,0.0,0.3,...,995.500000,970.770996,86.0,78.0,30.0,5.0,24140.0,0.03,0.000000,0.443371
1,mumbai,2025-06-23 21:00:00,27.615000,88.0,25.446417,30.897842,100.0,1.5,0.1,1.4,...,1000.700012,999.564087,100.0,82.0,100.0,8.0,15980.0,0.16,0.037777,0.443024
2,bengaluru,2025-06-23 21:00:00,21.063000,85.0,18.442924,21.985312,10.0,0.0,0.0,0.0,...,1007.400024,908.368225,99.0,60.0,79.0,89.0,24140.0,0.02,0.016859,0.374470
3,hyderabad,2025-06-23 21:00:00,25.013000,77.0,20.699442,26.771729,3.0,0.0,0.0,0.0,...,1002.200012,944.763489,100.0,70.0,100.0,100.0,24140.0,0.03,0.047400,0.728772
4,chennai,2025-06-23 21:00:00,26.726000,75.0,21.925547,30.430393,0.0,0.0,0.0,0.0,...,1002.900024,1001.644226,100.0,0.0,100.0,100.0,24140.0,0.08,0.024065,0.876472
5,kolkata,2025-06-23 21:00:00,27.056499,94.0,26.006813,33.903339,28.0,0.2,0.0,0.2,...,996.599976,995.466553,100.0,74.0,40.0,100.0,24140.0,0.02,0.000000,0.214381
6,ahmedabad,2025-06-23 21:00:00,26.806499,95.0,25.937366,33.244984,38.0,1.7,0.0,1.7,...,997.500000,991.500244,100.0,85.0,82.0,100.0,23140.0,0.03,0.000000,0.176052
7,pune,2025-06-23 21:00:00,23.871500,85.0,21.195686,26.976608,8.0,0.0,0.0,0.0,...,1002.400024,939.278198,100.0,100.0,79.0,100.0,24140.0,0.04,0.008580,0.443923
8,jaipur,2025-06-23 21:00:00,26.289499,92.0,24.884874,31.850639,75.0,0.7,0.0,0.7,...,996.400024,948.806824,92.0,85.0,33.0,41.0,23920.0,0.04,0.000000,0.273277
9,lucknow,2025-06-23 21:00:00,27.317501,88.0,25.153664,33.249401,53.0,0.0,0.0,0.0,...,995.799988,982.657349,87.0,75.0,50.0,0.0,24140.0,0.03,0.000951,0.435405
